## Collecting data from gnomAD

Variants data downloaded from [gnomAD](https://gnomad.broadinstitute.org/downloads#v4)

Use command line for initial processing

1. Filtered only PASS variants 

```zsh
bcftools view -f 'PASS,.' gnomad.exomes.v4.0.sites.chr22.vcf.bgz > filtered_gnomad22.bgz
```
For next parcing we can use also **command line** or **python**

2. Extract the necessary data

for this step we need **bcftools** utility

```zsh
bcftools query -f '%CHROM\t%POS\t%ID\t%REF\t%ALT\t%AC\t%AC_afr\t%AC_amr\t%AC_nfe
\t%AC_asj\t%AC_sas\t%AC_eas\t%AC_mid\t%AC_fin\t%AN\t%AN_afr\t%AN_amr\t%AN_nfe\t%AN_asj
\t%AN_sas\t%AN_eas\t%AN_mid\t%AN_fin\t%AF\t%AF_afr\t%AF_amr\t%AF_nfe\t%AF_asj\t%AF_sas
\t%AF_eas\t%AF_mid\t%AF_fin\t%vep=\n' filtered_gnomad22.bgz > processed_gnomad22.vcf.bgz
```


In [1]:
import numpy as np
import pandas as pd
import cyvcf2
import csv

In [2]:
# Input VCF.bgz file path
vcf_path = 'data/test.bgz'
output_csv_path = 'gnomad_data.csv'

# Define columns to extract
columns_to_extract = ['CHROM', 'POS', 'ID', 'REF', 'ALT']

# Define INFO fields to extract
info_fields_to_extract = ['AC', 'AC_afr', 'AC_amr', 'AC_nfe', 'AC_asj', 'AC_sas',
                         'AC_eas', 'AC_mid', 'AC_fin', 'AN', 'AN_afr', 'AN_amr', 'AN_nfe', 'AN_asj', 'AN_sas',
                         'AN_eas', 'AN_mid', 'AN_fin', 'AF', 'AF_afr', 'AF_amr', 'AF_nfe',
                         'AF_asj', 'AF_sas', 'AF_eas', 'AF_mid', 'AF_fin']

# Open VCF.bgz file using cyvcf2
vcf_reader = cyvcf2.VCF(vcf_path)

# Extract data from the VCF.bgz file
data = []
for variant in vcf_reader:
    variant_data = [variant.CHROM, variant.POS, variant.ID, variant.REF, variant.ALT[0]]
    info_data = [variant.INFO.get(field, '.') for field in info_fields_to_extract]
    vep_annotation = variant.INFO.get('vep')
    if vep_annotation:
        vep_info_combined = []
        for annotation in vep_annotation.split(','):
            vep_info = annotation.split('|')
            vep_info_combined.append(','.join([vep_info[i] if i < len(vep_info) else '' for i in [3, 1, 43, 44]]))
        data.append(variant_data + info_data + [','.join(vep_info_combined)])
    else:
        data.append(variant_data + info_data + [''])  # Adding empty values if VEP annotation is not present

# Save the extracted data into a CSV file
with open(output_csv_path, 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    header = columns_to_extract + info_fields_to_extract + ['VEP_annotation']
    writer.writerow(header)
    writer.writerows(data)

print(f"Extracted data has been saved to {output_csv_path}")

Extracted data has been saved to gnomad_data.csv


In [3]:
var_data = pd.read_csv('gnomad_data.csv', sep = ',')  
display(var_data)

,CHROM,POS,ID,REF,ALT,AC,AC_afr,AC_amr,AC_nfe,AC_asj,...,AF,AF_afr,AF_amr,AF_nfe,AF_asj,AF_sas,AF_eas,AF_mid,AF_fin,VEP_annotation
0,chr22,10736362,rs879131185,C,T,63,0,1,50,0,...,1.944440e-01,0.0,0.5,0.19841299951076508,.,0.0,0.16666699945926666,0.5,0.20000000298023224,"U2,upstream_gene_variant,,"
1,chr22,10736399,rs1206833908,T,C,442,13,5,298,5,...,4.900220e-01,0.5416669845581055,0.5,0.48534199595451355,0.5,0.5,0.4375,0.5,0.5,"U2,upstream_gene_variant,,"
2,chr22,10736400,rs1289580688,G,A,15,0,0,9,0,...,1.500000e-02,0.0,0.0,0.013274299912154675,0.0,0.0,0.0,0.03846149891614914,0.022727299481630325,"U2,upstream_gene_variant,,"
3,chr22,10736414,NaN,G,C,1,0,0,1,0,...,7.194240e-04,0.0,0.0,0.0010019999463111162,0.0,0.0,0.0,0.0,0.0,"U2,upstream_gene_variant,,"
4,chr22,10736415,NaN,C,T,1,0,0,1,0,...,6.729480e-04,0.0,0.0,0.0009191179997287691,0.0,0.0,0.0,0.0,0.0,"U2,upstream_gene_variant,,"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2364,chr22,15562669,rs565181402,C,G,8,0,0,0,0,...,6.055540e-06,0.0,0.0,0.0,0.0,9.12670002435334e-05,0.0,0.00021987699437886477,0.0,",intron_variant&non_coding_transcript_variant,..."
2365,chr22,15562671,rs1246240081,C,T,6,0,0,5,0,...,4.729080e-06,0.0,0.0,5.280080131342402e-06,0.0,1.2872000297647901e-05,0.0,0.0,0.0,",intron_variant&non_coding_transcript_variant,..."
2366,chr22,15562676,NaN,T,C,2,0,0,2,0,...,1.630470e-06,0.0,0.0,2.1937898964097258e-06,0.0,0.0,0.0,0.0,0.0,",intron_variant&non_coding_transcript_variant,..."
2367,chr22,15562681,NaN,C,A,1,0,0,0,0,...,9.840270e-07,0.0,0.0,0.0,0.0,0.0,2.762889926088974e-05,0.0,0.0,",intron_variant&non_coding_transcript_variant,..."


In [10]:
# Not good idea

var_data = pd.read_csv('gnomad_data.csv', sep = ',')  

vep_split = var_data['VEP_annotation'].str.split(',', expand=True)
# Create new column names
new_column_names = [f'VEP_annotation_{i+1}' for i in range(vep_split.shape[1])]

# Rename the new columns to meaningful names
vep_split.columns = new_column_names

# Fill missing values with NaN
#vep_split = vep_split.where(vep_split.ne(''), np.nan)

# Concatenate the original DataFrame with the new columns
var_data1 = pd.concat([var_data, vep_split], axis=1)

display(var_data['VEP_annotation'])
display(var_data1['VEP_annotation_4'].unique())

0                              U2,upstream_gene_variant,,
1                              U2,upstream_gene_variant,,
2                              U2,upstream_gene_variant,,
3                              U2,upstream_gene_variant,,
4                              U2,upstream_gene_variant,,
                              ...                        
2364    ,intron_variant&non_coding_transcript_variant,...
2365    ,intron_variant&non_coding_transcript_variant,...
2366    ,intron_variant&non_coding_transcript_variant,...
2367    ,intron_variant&non_coding_transcript_variant,...
2368    ,intron_variant&non_coding_transcript_variant,...
Name: VEP_annotation, Length: 2369, dtype: object

array(['', 'HC'], dtype=object)